## Made by -----> David Laessker - Peter Fagrell <------



### Perceptron

Since the perceptron is a linnear classifier it can only classify data that is linearly seperable. This means that it can only classify data that can be seperated by a line. We can represent this very arcaheicly by the following:

Trainig data 1 would look like this 

x|x
-|-
x|o

and training data 2 would look like this

o|x
-|-
x|o

The top two boxes are Gothenburg/Sydney bottom two are Paris,  the right boxes are July and the left boxes are December.


As we can see we can easily draw a line that separates the two classes in example 1 but not in example 2. This means that the perceptron can only classify example 1 and not example 2.


### Accuracy 

- PegasosLREG = 0.8309

- PegasosSVC = 0.8431





In [19]:
import numpy as np
from sklearn.base import BaseEstimator
from aml_perceptron import LinearClassifier

Our implemetation

In [21]:
class PegasosSVC(LinearClassifier):
    """
    Implementation of the Pegasos algorithm for SVCs.
    """

    def __init__(self, lambda_reg=0.1, n_iter=100000):
        self.lambda_reg = lambda_reg
        self.n_iter = n_iter

    def fit(self, X, Y):

        self.find_classes(Y)
        Y_encoded = self.encode_outputs(Y)

        if not isinstance(X, np.ndarray):
            X = X.toarray()

        n_features = X.shape[1]
        self.w = np.zeros(n_features)
        self.lambda_reg = 1/n_features

        for t in range(1, self.n_iter):
            rand = np.random.randint(0, len(X))
            x, y = X[rand], Y_encoded[rand]

            n = 1/(self.lambda_reg*t)

            score = x.dot(self.w)

            if y*score <= 1:
                self.w = (1 - n*self.lambda_reg) * self.w + n*y*x
            else:
                self.w = (1 - n*self.lambda_reg) * self.w


class PegasosLREG(LinearClassifier):
    """
    Implementation of the Pegasos algorithm for logistic regression.
    """

    def __init__(self, lambda_reg=0.1, n_iter=100000):
        self.lambda_reg = lambda_reg
        self.n_iter = n_iter

    def fit(self, X, Y):
        self.find_classes(Y)
        Y_encoded = self.encode_outputs(Y)

        if not isinstance(X, np.ndarray):
            X = X.toarray()

        n_features = X.shape[1]
        self.w = np.zeros(n_features)
        self.lambda_reg = 1/n_features

        for t in range(1, self.n_iter):
            rand = np.random.randint(0, len(X))
            x, y = X[rand], Y_encoded[rand]

            n = 1/(self.lambda_reg*t)

            score = x.dot(self.w)
            gradient = -y * x * (1 - 1/(1 + np.exp(-y * score)))

            self.w = (1 - n * self.lambda_reg) * self.w - n * gradient
            self.w *= min(1, 1 / (np.sqrt(self.lambda_reg)
                          * np.linalg.norm(self.w)))


Make class

In [16]:
class Perceptron(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        # Perceptron algorithm:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):

                # Compute the output score for this instance.
                score = x.dot(self.w)

                # If there was an error, update the weights.
                if y*score <= 0:
                    self.w += y*x